# Volumetric Stiffness Field Playground

In [1]:
# Auto reload modules
%load_ext autoreload
%autoreload 2

#if you haven't installed vsf and just want to run this from the current directory, uncomment the following lines
import sys
sys.path.append('..')

from vsf import vsf_from_file
from klampt.io import open3d_convert
from klampt import vis, Geometry3D, WorldModel
from vsf.visualize.klampt_visualization import vsf_show

vis.init('PyQt')  #needed to pop up Klampt OpenGL windows in Jupyter notebook
None

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
***  klampt.vis: using Qt6 as the visualization backend  ***


### Download the data from box

+ #### Download single test case: 
    + Demo data: https://uofi.box.com/s/31wozq63qgqvg8012r1jdfj5mdchmmfp

+ #### Download the full dataset
    + Download objects assets: https://uofi.box.com/s/2wj11zze45nj2owwqdpm7tsh0keleedw
    + Download estimates VSF models for simulation: https://uofi.box.com/s/g1i02pxf4quk0zt7nzv4qhovlhfnt3bl
    + Download raw visual-tactile dataset: https://uofi.box.com/s/grqsjh5m27jkin14t097pvjvy9b1bk6r

## Loading a VSF from file

In this example we will load one of the demo objects.  Let's assume that you have placed `demo_data` in the parent directory of this Jupyter notebook.

In [2]:
vsf_model = vsf_from_file('../demo_data/est_rubber_fig_tall_angle00/final_est')
print('Model loaded')

Model loaded


Now we can visualize the loaded model.  We can do so in Open3D using a point cloud visualization, or in Klampt with a point cloud or implicit surface visualization.

In [3]:
# Visualize VSF point cloud in Open3D
import open3d as o3d
from vsf.visualize.o3d_visualization import vsf_to_point_cloud
vis_pcd = vsf_to_point_cloud(vsf_model)
o3d.visualization.draw_geometries([vis_pcd], window_name='VSF vis_pcd')

Stiffness range:  -2.4917295937509232e-17 106.25068228144467


In [4]:
# Klampt point cloud visualization of VSF
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
k_pcd = Geometry3D(vsf_to_point_cloud(vsf_model))
vis.debug(k_pcd)

Stiffness range:  -2.4917295937509232e-17 106.25068228144467
################################################################
klampt.vis: Running multi-threaded dialog, waiting to complete...
vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 1 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no action menu?
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?


#########################################
klampt.vis: Dialog done on window 1 result 1
#########################################
klampt.vis: ... dialog done, leaving thread open
################################################################
vis: creating GL window
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Making window 0
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWindow.resizeGL: called when invisible?
QtGLWindow.paintGL: called while invisible?
#########################################
klampt.vis: Window 0 close
#########################################
vis: creating GL window
######### QtGLWindow setProgram ###############
######### QtGLWindow setProgram ###############
#########################################
klampt.vis: Dialog on window 2 starting
#########################################
######### QtGLWindow Initialize GL ###############
QtGLWidget.initialize: no ac

In [5]:
# Klampt volumetric visualization of VSF
stiffness_levels = [0.27072093, 4.54502328, 8.81932563, 9.09362797, 10.36793032]
vsf_show(vsf_model, stiffness_levels)

Setting voxel grid resolution 0.020093083968962675
klampt.vis: auto-fitting camera to scene.


## VSF estimation from tactile data

In this example, we will create VSF from a dataset that includes tactile observations.

- Create an empty VSF.  We will use a VSFFactory to shape the VSF rest points from an RGBD point cloud
- Configure the robot model and sensor(s).  In this case the robot is a Kinova Gen3, and there is only one sensor that provides joint torques.
- Load a dataset of trials giving robot actions and observations from sensor(s) 
- Create an estimator, and run it in batch mode on the dataset.

Let's start by creating an empty VSF.  If you have an RGB-D scanned object, you should use a `VSFRGBDCameraFactory` like the following code to construct the VSF points from a region of interest in the point cloud, extending through a volume behind the camera's point of view.  It doesn't really matter how you get the points; you can also create an empty box using the `vsf.vsf_from_box()` function, create a VSF from a 3D scanned mesh using `vsf.vsf_from_mesh()`, or load a previously saved VSF from disk.

In [6]:
from vsf.core.vsf_factory import VSFRGBDCameraFactory, VSFRGBDCameraFactoryConfig, ViewConfig

view = ViewConfig(origin=[-0.55639158,1.04689234,0.52593784])  # need to set the origin of the camera so that volume points can be created
#we will select points within a bounding box 
config = VSFRGBDCameraFactoryConfig(fps_num=20000, voxel_size=0.02,
                                    view = view, bbox = [[-1.03, -0.64, -0.22], [-0.29, 0.23, 0.73]], downsample_visual=True, verbose=True)
factory = VSFRGBDCameraFactory(config)
#process() does all the work
vsf_empty = factory.process('../demo_data/rubber_fig_tall_angle00/bg_pcd.pcd')  #the PCD file is the point cloud of the RGBD scene, including the background 
print('{} rest points for VSF model created from RGBD factory'.format(len(vsf_empty.rest_points)))

BBox selection: keeping 129694 points out of 459947
downsampling visual point cloud to voxel size 0.02
iter step 0 left pts: 129694
iter step 1 left pts: 129666
iter step 2 left pts: 129567
iter step 3 left pts: 129479
iter step 4 left pts: 129241
iter step 5 left pts: 129014
iter step 6 left pts: 128862
iter step 7 left pts: 128657
iter step 8 left pts: 128345
iter step 9 left pts: 128086
iter step 10 left pts: 127695
iter step 11 left pts: 127290
iter step 12 left pts: 126953
iter step 13 left pts: 126644
iter step 14 left pts: 126349
iter step 15 left pts: 126033
iter step 16 left pts: 125654
iter step 17 left pts: 125288
iter step 18 left pts: 124893
iter step 19 left pts: 124478
iter step 20 left pts: 124064
iter step 21 left pts: 123677
iter step 22 left pts: 123282
iter step 23 left pts: 122795
iter step 24 left pts: 121882
iter step 25 left pts: 120518
iter step 26 left pts: 118863
iter step 27 left pts: 117332
iter step 28 left pts: 114992
iter step 29 left pts: 110272
iter st

In [7]:
# Show the processed VSF model
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(empty_pc = vsf_to_point_cloud(vsf_empty, masked_view_fraction=1.0))

Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 2
#########################################
vis.dialog(): waiting for window 2 on vis thread to complete....
vis.dialog(): ... dialog done


Next, we create a world with a robot, a simulator, and one or more sensors.  This example loads a Kinova Gen3 robot model and uses the joint torque sensors in the simulator.  We also add the empty VSF into the simulator using `sim.add_deformable`.  This creates a body that can be moved through space.  (By default, all VSFs are considered to have a static pose, which works well if the object doesn't move much during interaction.  We will consider moving objects later.)

In [8]:
from vsf.sim import klamptWorldWrapper, QuasistaticVSFSimulator
from vsf.sim.point_vsf_body import ContactParams
from vsf.sensor.joint_torque_sensor import JointTorqueSensor

#create a world
world = klamptWorldWrapper()
world.add_robot('kinova','../knowledge/robot_model/kinova_gen3_repaired.urdf')
robot = world.world.robot(0)

#this preprocessing needs to be done before running the simulator with a point VSF body
world.setup_local_pcd_lst('open3d')
world.setup_local_sdf_lst()

#create a simulator with the world, a joint torque sensor, and the vsf body
sensor = JointTorqueSensor('kinova_joint_torques','kinova',[robot.link(i).name for i in range(1,8)])
sim = QuasistaticVSFSimulator(world, [sensor])
vsf_body = sim.add_deformable('vsf',vsf_empty, contact_params=None)  #if you want to customize how the VSF is simulated, you can pass in a ContactParams object here

WorldModel::LoadRobot: ../knowledge/robot_model/kinova_gen3_repaired.urdf
URDFParser: Link size: 9
URDFParser: Joint size: 9
URDFParser: Done loading robot file ../knowledge/robot_model/kinova_gen3_repaired.urdf
FMM_Fill identifies 1305 surface, 731 interior, 2003 exterior cells
FMM starting with 835 surface cells, grid of size 13 13 21
FMM found 1149 interior and 2400 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.0648063 -0.0649216 -0.019575   0.0651937 0.0650784 0.190425
FMM_Fill identifies 1076 surface, 539 interior, 1737 exterior cells
FMM starting with 684 surface cells, grid of size 13 13 21
FMM found 864 interior and 2685 exterior cells
Geometry: AnyGeometry::Convert: FMM grid bounding box -0.065 -0.0662972 -0.193375   0.065 0.0637028 0.016625
FMM_Fill identifies 1503 surface, 788 interior, 2376 exterior cells
FMM starting with 829 surface cells, grid of size 13 31 13
FMM found 1113 interior and 4126 exterior cells
Geometry: AnyGeometry::Convert: FMM gr

In [9]:
# show the world and VSF
from vsf.visualize.klampt_visualization import vsf_to_point_cloud
vis.debug(world=world.world, empty_pc = vsf_to_point_cloud(vsf_empty), origin = view.origin)

Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 3
#########################################
vis.dialog(): waiting for window 3 on vis thread to complete....
TriMeshTopology: mesh has 9 triangles with duplicate neighbors!
  Triangle range 273 to 924
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 104 triangles with duplicate neighbors!
  Triangle range 3828 to 4274
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 271 triangles with duplicate neighbors!
  Triangle range 80 to 3362
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 635 triangles with duplicate neighbors!
  Triangle range 29 to 3626
  May see strange results for some triangle mesh operations
TriMeshTopology: mesh has 587 triangles with duplicate neighbors!
  Triangle range 1988 to 3472
  May see strange results for some triangle mesh operations
TriMeshTopolo

Now we will set up a dataset. We will load a dataset from a standard folder format, which is configured and loaded automatically in the following example.  We will use a `DatasetConfig` which specifies which keys are present, and which keys used for robot commands and sensor observations.  The `dataset_from_config` function will set up a dataset appropriately to follow the specified configuration.  The standard dataset lazy-loads sequences into memory.

More generally, a dataset is any object that can be treated like a list of sequences, and each sequence is a list of dictionaries mapping keys to numpy arrays.  In other words, datasets behave like a `List[List[Dict[str,np.ndarray]]]` type. If you don't want to go through the trouble of saving such a thing to disk, you can just create such an object yourself.  

In [10]:
from vsf.dataset.constructors import DatasetConfig, dataset_from_config

keys = {'torques':7,'angles':7}  #describes the keys present in the dataset
dataset_config = DatasetConfig('../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm',
                               keys,
                               sensor_keys={'kinova_joint_torques':'torques'},
                               control_keys={'kinova':'angles'})
dataset = dataset_from_config(dataset_config)
print("Dataset has {} sequences".format(len(dataset)))

Dataset has 32 sequences


For debugging, let's just validate the trajectories coming from the dataset.  Here we'll show the first 5 trajectories.

In [11]:
from klampt.model.trajectory import RobotTrajectory 

for seqno in range(len(dataset)):
    #extract the sequence of commands
    seq = dataset[seqno]
    commands = []
    for frame in range(len(seq)):
        frame = seq[frame]
        commands.append(frame['angles'])

    #convert to a RobotTrajectory and show it
    configs = [robot.configFromDrivers(d) for d in commands]
    traj = RobotTrajectory(robot,[i/len(configs) for i in range(len(configs))],configs)
    vis.debug(robot, {'animation':traj}, 'empty_pc', vsf_to_point_cloud(vsf_empty))
    if seqno >= 4: break

Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm/seq_1704083702
Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 4
#########################################
vis.dialog(): waiting for window 4 on vis thread to complete....
vis.dialog(): ... dialog done
Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm/seq_1704083759
Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 5
#########################################
vis.dialog(): waiting for window 5 on vis thread to complete....
vis.dialog(): ... dialog done
Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm/seq_1704083799
Stiffness range:  0.0 0.0
#########################################
klampt.vis: Creating dialog on window 6
#########################################
vis.dialog(): waiting for window 

We can also check the discrepancy between the observations and the predicted measurements for a random VSF stiffness.  Here, we're also introducing the notion of sensor calibration, which is usually necessary at the beginning of a trial to tare the values coming from the sensor.

A `BaseSensor` can potentially handle its own calibration, but we find it's easier to separate how to calibrate the sensor from the sensor simulator itself.  Here we will generate a `BaseCalibrator` object that we will run at the beginning of each trial to calibrate the sensor.

(There is a little bit of work here to convert the dataset keys to the control and sensor keys expected by the simulator, calibrator, and estimators. This boilerplate is used a lot...)

In [12]:
from vsf.sensor import TareCalibrator
import numpy as np

vsf_empty.stiffness.fill_(0.1)  #set a guessed stiffness of the VSF model
calibrator = TareCalibrator()
for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    control_seq = []
    sensor_seq = []
    for frame in seq:
        control_seq.append({k:frame[v] for k,v in dataset_config.control_keys.items()})
        sensor_seq.append({k:frame[v] for k,v in dataset_config.sensor_keys.items()})
        
    #run the calibration
    sim.reset()
    n = calibrator.calibrate(sensor,sim,control_seq,sensor_seq)
    #returns the # of samples used in calibration.  Technically should skip this number of frames for estimation
    print("Sequence",seqno,"calibration:",sensor.get_calibration())

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    diffs = []
    for frameno in range(n,len(seq)):
        sim.step(control_seq[frameno],dt)
        pred = sim.measurements()['kinova_joint_torques']
        actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(pred) == len(actual)
        diffs.append(pred-actual)
    diffs = np.array(diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(diffs**2,axis=0)))

    if seqno >= 4: break


Sequence 0 calibration: {'tare': array([ 0.45979138, 18.23648429,  0.42123438,  2.86159472,  0.06669243,
       -0.8061054 , -0.30282573]), 'gravity_pred_at_tare': array([-1.35915481e-04,  1.91127152e+01,  2.38662322e-01,  2.89546379e+00,
        5.89215163e-02, -9.54574667e-01, -7.94720591e-03])}
Sequence 0 joint torque RMSEs [9.91913453 3.20994491 4.95261744 1.81149835 0.49278865 0.48767311
 0.09661432]
Sequence 1 calibration: {'tare': array([ 0.52451001, 19.72127437, -0.08059071,  3.76032879,  0.34009876,
       -0.71614274, -0.38496185]), 'gravity_pred_at_tare': array([ 4.59781161e-01,  1.96244829e+01, -3.01500764e-03,  4.47961341e+00,
        3.32438221e-01, -7.43565164e-01, -2.97378133e-01])}
Sequence 1 joint torque RMSEs [5.87719267 2.9586089  3.12015592 0.66501139 0.89273961 0.58525534
 0.26543169]
Sequence 2 calibration: {'tare': array([ 0.37255337, 18.40848739,  0.55852668,  3.35344802,  0.42691417,
       -0.49719183, -0.20882375]), 'gravity_pred_at_tare': array([ 0.06459304

Now, let's start the estimation process.  Let's start by using *batch* estimation, which uses all the items in the dataset to estimate the VSF stiffness parameters.  We have to provide a prior to the estimator to initialize its stiffness guess, and we will use a GaussianVSFPriorFactory which assigns an independent Gaussian distribution to each point's stiffness.

We can also customize the configuration of the estimator, such as the optimization technique used, but let's not worry about this too much for now.

In [16]:
from vsf.sensor import TareCalibrator
from vsf.estimator.point_vsf_estimator import PointVSFEstimator, PointVSFEstimatorConfig
from vsf.prior.prior_factory import GaussianVSFPriorFactory
import time
import copy
vsf_est = vsf_empty
vsf_empty = copy.deepcopy(vsf_est)  #save a copy of the uninitialized VSF for later

#the second argument sets the prior estimate to a mean of 0.5 and a standard deviation of 1.0
estimator = PointVSFEstimator(PointVSFEstimatorConfig(), GaussianVSFPriorFactory(0.5,1.0))

t0 = time.time()
estimator.batch_estimate(sim, vsf_est, [dataset[i] for i in range(5)], dataset_config, {sensor.name:TareCalibrator()})
t1 = time.time()
print("Estimation took time",t1-t0)

Generating simulation cache
Number of observations: 5
Observed indices: 1040


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "


QuadProgOptimizer: start solving cvxpy problem...
QuadProgOptimizer: cvxpy solve time = 3.317596197128296
Estimation took time 11.138893842697144


Now let's see some statistics about the estimates.  The VSF stiffness values are in the `stiffness` attribute, and it will also have new two features, 'K_std' and 'N_obs', which give the standard deviations of the estimates and the number of frames in which each point has been touched, respectively.

In [17]:
print('Features:',list(vsf_est.features.keys()))
stiffness = vsf_est.stiffness.cpu().numpy()
N_obs = vsf_est.features['N_obs'].cpu().numpy()
touched_mask = N_obs > 0
print('Stiffness min {}, max {}, mean {}, std {}'.format(np.min(stiffness),np.max(stiffness),np.average(stiffness),np.std(stiffness)))
print('Number of points touched {}/{}'.format(np.sum(N_obs > 0),stiffness.shape[0]))
print('Touched stiffness min {}, max {}, mean {}, std {}'.format(np.min(stiffness[touched_mask]),np.max(stiffness[touched_mask]),np.average(stiffness[touched_mask]),np.std(stiffness[touched_mask])))


Features: ['K_std', 'N_obs']
Stiffness min -2.726397961849863e-20, max 28.396162033081055, mean 0.5455399751663208, std 0.6045356392860413
Number of points touched 1040/20000
Touched stiffness min -2.726397961849863e-20, max 28.396162033081055, mean 1.3757702112197876, std 2.5101892948150635


Now let's visualize the estimates. For clearer visualization, we will set the stiffness of the untouched points to 0.

In [18]:
vsf_est.stiffness[~touched_mask] = 0
vis.debug(world=world.world, estimated_pc = vsf_to_point_cloud(vsf_est), origin = view.origin)

Stiffness range:  -2.726398e-20 28.396162
#########################################
klampt.vis: Creating dialog on window 9
#########################################
vis.dialog(): waiting for window 9 on vis thread to complete....
vis.dialog(): ... dialog done


Finally, let's check the accuracy of the predictions on the 5-sequence training set.

In [19]:
for seqno in range(len(dataset)):
    #extract the sequence of controls and observations.  This is boilerplate
    seq = dataset[seqno]
    control_seq = []
    sensor_seq = []
    for frame in seq:
        control_seq.append({k:frame[v] for k,v in dataset_config.control_keys.items()})
        sensor_seq.append({k:frame[v] for k,v in dataset_config.sensor_keys.items()})
        
    #run the calibration
    sim.reset()
    n = calibrator.calibrate(sensor,sim,control_seq,sensor_seq)

    #now, run the simulator and compare the predicted torques to the actual torques
    dt = 0.1  # a guessed time step.  There's no time-dependent functionality in the quasistatic simulator, so this doesn't matter
    diffs = []
    for frameno in range(n,len(seq)):
        sim.step(control_seq[frameno],dt)
        pred = sim.measurements()['kinova_joint_torques']
        actual = sensor_seq[frameno]['kinova_joint_torques']
        assert len(pred) == len(actual)
        diffs.append(pred-actual)
    diffs = np.array(diffs)
    print("Sequence",seqno,"joint torque RMSEs",np.sqrt(np.mean(diffs**2,axis=0)))

    if seqno >= 4: break

Sequence 0 joint torque RMSEs [0.73303994 3.80165088 1.03696311 1.99729094 0.70382661 0.39356196
 0.12847682]
Sequence 1 joint torque RMSEs [2.35298485 1.16985833 1.37596327 0.99714673 0.73072358 0.43228238
 0.26002846]
Sequence 2 joint torque RMSEs [3.80693566 0.68984184 1.4025449  0.94029851 0.2939409  0.18091059
 0.28451421]
Sequence 3 joint torque RMSEs [1.87975762 0.91273379 1.31222036 0.9972302  0.32634134 0.46176164
 0.20817396]
Sequence 4 joint torque RMSEs [0.28476413 0.41272718 0.29000763 0.770336   0.11368901 0.18179088
 0.22560368]


Since comparisons are done frequently, we also have standard functions for calculating these metrics.

In [22]:
from vsf.sim.metrics import rmse_sensors
calibrators = {'kinova_joint_torques':calibrator}
rmses = rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators)
for i in range(5):
    print("Sequence",i,"joint torque RMSEs",rmses['kinova_joint_torques'][i])
sim.vsf_objects['vsf'].vsf_model = vsf_empty
print("Original RMSE train",rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
print("Original RMSE test",rmse_sensors(sim,[dataset[i] for i in range(5,30)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
sim.vsf_objects['vsf'].vsf_model = vsf_est
print("Estimated RMSE train",rmse_sensors(sim,[dataset[i] for i in range(5)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])
print("Estimated RMSE test",rmse_sensors(sim,[dataset[i] for i in range(5,30)],dataset_config,calibrators,aggregate_seqs=True,aggregate_channels=True)['kinova_joint_torques'])


Sequence 0 joint torque RMSEs [0.7493814  3.79082726 0.84790059 2.06831264 0.71367177 0.91755065
 0.20167259]
Sequence 1 joint torque RMSEs [2.43529267 1.18730576 1.57005884 1.08928811 0.72808527 0.67161187
 0.3530975 ]
Sequence 2 joint torque RMSEs [3.92197014 0.70663039 1.41258548 1.14950111 0.29892019 0.50347354
 0.19000611]
Sequence 3 joint torque RMSEs [1.87975762 0.91273379 1.31222036 0.9972302  0.32634134 0.46176164
 0.20817396]
Sequence 4 joint torque RMSEs [0.28476413 0.41272718 0.29000763 0.770336   0.11368901 0.18179088
 0.22560368]
Original RMSE train 2.6640317962954736
Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm/seq_1704084155
Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm/seq_1704084218
Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_trail1/arm/seq_1704084271
Loading sequence ../demo_data/kinova_joint_torques_dataset/rubber_fig_tall_angle00_t

Now we see that we've done a much better job of estimating the RMSEs on the training set but we haven't really improved the testing RMSEs very much.  This is because we've only used 5 sequences for training, and they may have not done a good job of covering the object.  This is a typical issue with using touch data alone.  Read on to the next section to explore better ways of extrapolating using vision features and priors. 

## Extrapolating VSF data by color

## Estimating VSFs for bodies that move

TODO: will need to load the pose from the dataset for each frame, update `vsf_body.pose`.

## Estimating VSFs for hand-held tactile sensors

TODO: will need to create an empty rigid body in the `klamptWorldWrapper` and attach the tactile sensor to the body. 

## Multi-sensor estimation

The estimation techniques above can work for any number of sensors.  Simply create more sensors to add to the simulator, and set up the observations dictionary to the estimator appropriately.    